# Workshop - MELI Data Challenge 2021

In [ ]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import csv
import pickle
import matplotlib.pyplot as plt
import multiprocessing as mp
from itertools import chain, islice
from datetime import timedelta
import jsonlines
import seaborn as sns
from pathlib import Path
import core.evaluators.metrics as metrics
import multiprocessing as mp
from itertools import chain, islice
import gzip

### 1. Fetching the data

#### Load train and test datasets

In [ ]:
# set up the directory where the challenge data is stored
data_dir = Path('../data')

In [ ]:
data_train = pd.read_parquet(data_dir/'train_data.parquet')
data_test = pd.read_csv(data_dir/'test_data.csv')

In [ ]:
data_train.head()

In [ ]:
data_test.head()

#### Load extra item data

In [ ]:
### auxiliary function to read jsonlines files
def load_jsonlines(filename):
    
    rv = []
    for obj in tqdm(jsonlines.open(filename)):
        rv.append(obj)
    return rv

In [ ]:
item_metadata = load_jsonlines(data_dir/'items_static_metadata_full.jl')

#### Convert to a df and use sku as the index

In [ ]:
df_metadata = pd.DataFrame(item_metadata)
df_metadata.index = df_metadata.sku
df_metadata.drop(columns=['sku'],inplace=True)

In [ ]:
df_metadata.head()

#### Hydrate the initial datasets with the extra data

In [ ]:
data_train = data_train.join(df_metadata, on='sku',how='left')

In [ ]:
data_test = data_test.join(df_metadata, on='sku',how='left')

In [ ]:
data_train.head(3)

In [ ]:
data_test.head()

### 2. Exploration

#### List all the columns 

In [ ]:
for col in data_train.columns:
    print(col)

#### Get some stats for each column

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
def describe_cols(cols,df):
    
    for col in cols: 
        print('\t COLUMN: ', col)
        print('\t type: ', df[col].dtype,'\n')
        print(df[col].describe(),'\n')

In [ ]:
columns_to_describe = ['date','listing_type','current_price']

In [ ]:
describe_cols(columns_to_describe,data_train)

### Visualize the time series

#### Visualize daily sales grouped by site

In [ ]:
# First we summarize the info
summary_site = data_train.groupby(['site_id','date']).sold_quantity.sum().reset_index()
summary_site.head()

In [ ]:
def plot_time_series(summary_data,time_var,series,level):
    
    plt.figure(figsize=(15, 4))
    plt.title(f'{series} time series grouped by {level}')
    sns.lineplot(data=summary_data, 
                 x=time_var,y=series,hue=level)
    plt.xticks(rotation=45)
    plt.show()
    

In [ ]:
# Then we plot it
plot_time_series(summary_site, time_var='date',series='sold_quantity',level='site_id')

#### Visualize weekly sales grouped by site

In [ ]:
# Define a new variable based on the date column to extract the week number
data_train['week'] = pd.to_datetime(data_train.date).dt.week

In [ ]:
# Summarize info
summary_site_w = data_train.groupby(['site_id','week']).sold_quantity.sum().reset_index()

In [ ]:
# Then we plot it
plot_time_series(summary_site_w,time_var='week',series='sold_quantity',level='site_id')

#### Get the top levels of categorical variable for a site 

In [ ]:
def get_top_categories(df, categorical_var, site_id, by, N=10):
    
    grand_total = df[df.site_id == site_id][by].sum()
  
    top_cat_df = (df[df.site_id == site_id]
         .groupby(['site_id',categorical_var])[by]
         .sum()
         .sort_values(ascending=False)
         .head(N))
    
    top_cat_df = top_cat_df.reset_index()
    top_cat_df[f'relative_{by}'] = top_cat_df[by]/grand_total 
    
    return(top_cat_df[[categorical_var,by,f'relative_{by}']])

In [ ]:
top_domains_MLM = get_top_categories(data_train, 
                                     categorical_var= 'item_domain_id',
                                     site_id='MLM', 
                                     by='sold_quantity', 
                                     N=10)
top_domains_MLM

#### Asses overlap between train and test skus

In [ ]:
# library
import matplotlib.pyplot as plt
from matplotlib_venn import venn2

In [ ]:
def asses_overlap(df_train, df_test, key):
    
    figure, axes = plt.subplots(1, len(df_train.site_id.unique()),figsize=(16, 6))

    for i,site in enumerate(df_train.site_id.unique()):

        unique_train = df_train[df_train.site_id == site][key].unique()
        unique_test = df_test[df_test.site_id == site][key].unique()

        v = venn2(subsets=[set(unique_train),set(unique_test)],
                  set_labels = (f"Train \n ({len(unique_train)})", 
                        f"Test \n ({len(unique_test)}) "),
                  ax=axes[i],
                  set_colors=('purple', 'skyblue'), alpha = 0.6)
        axes[i].set_title(site)
    plt.show()
    

In [ ]:
asses_overlap(data_train, data_test, key='sku')

#### Plot distributions

##### Plot distribution for continuos variable

In [ ]:
site_id = 'MLM'
item_domain_id = 'MLM-CELLPHONE_COVERS'
#product_id = 'MLM15586828'
subset_data = data_train[(data_train.site_id == site_id)& (data_train.item_domain_id == item_domain_id)]


In [ ]:
subset_data.current_price.hist(bins=100)

##### Plot distribution for categorical variable

In [ ]:
subset_data.shipping_logistic_type.value_counts(normalize=True).plot.bar()

#### Plot the relationship between two continuos variables

In [ ]:
site_id = 'MLM'
item_domain_id = 'MLM-CELLPHONE_COVERS'
subset_data = data_train[(data_train.site_id == site_id)& (data_train.item_domain_id == item_domain_id)]

In [ ]:
def plot_bivariate(data,level, x, y, agg_x, agg_y):
    
    sns.scatterplot(data=data.groupby(level).agg(
        {x: agg_x,y: agg_y}),
                    x=x,y=y)
    plt.show()
    

In [ ]:
plot_bivariate(subset_data,
               x='current_price',
               level='sku',
               y='sold_quantity', 
               agg_x=np.mean, 
               agg_y=np.sum)

In [ ]:
plot_bivariate(subset_data,
               level='sku',
               x='minutes_active',
               y='sold_quantity', 
               agg_x=np.mean, 
               agg_y=np.sum)

#### Distribution of target stock

In [ ]:
figure, axes = plt.subplots(1, 2,figsize=(14, 6))
figure.suptitle('Distribution of target stock')
sns.histplot(x=data_test.target_stock,bins=5000, kde=False, ax=axes[0])
axes[0].set_xlim(0,80)
sns.boxplot(x=data_test.target_stock, ax=axes[1])
axes[1].set_xlim(0,80)
plt.show()

### 3. Building your validation set

In [ ]:
data_train.date.min(), data_train.date.max()

##### Make a temporary split

In [ ]:
split_date = (pd.to_datetime(data_train.date).max()-timedelta(days=30)).date()
print(split_date)

In [ ]:
#separete the last 30 days for validation
data_val = data_train.loc[(data_train.date > str(split_date))]

#use the rest as training
data_train = data_train.loc[(data_train.date <= str(split_date))]

##### Now let's build the validation dataset by calculating target stock and inventory days.

In [ ]:
#disclaimer: this is not the code that was used to generate the test_set.
# It was made from scratch

def create_validation_set(dataset):
    np.random.seed(42)
    print('Sorting records...')
    temp_pd = dataset.loc[:, ['sku','date','sold_quantity']].sort_values(['sku','date'])

    print('Grouping quantity...')
    temp_dict = temp_pd.groupby('sku').agg({'sold_quantity':lambda x: [i for i in x]})['sold_quantity'].to_dict()

    result = []
    for idx, list_quantity in tqdm(temp_dict.items(), desc='Making targets...'):
        cumsum = np.array(list_quantity).cumsum()
        stock_target = 0
        if cumsum[-1] > 0 and len(cumsum)==30:
            
            #choose a random target different from 0
            while stock_target == 0:
                stock_target = np.random.choice(cumsum)
                
            #get the first day with this amounnt of sales
            day_to_stockout = np.argwhere(cumsum==stock_target).min() + 1
            
            #add to a list
            result.append({'sku':idx, 'target_stock':stock_target, 'inventory_days':day_to_stockout})
    return result

#generate target for the 30 days of validation
val_dataset = create_validation_set(data_val)

In [ ]:
val_dataset[:10]

In [ ]:
y_true_val = [x['inventory_days'] for x in val_dataset]

### 4. Modeling

#### Baseline #1: UNIFORM distribution

We need a baseline to know what is our starting point. We will use it latter to validate more complex models.  
Besides we could iterate a simple baseline model to get better models

In [ ]:
days_to_predict = 30

In [ ]:
y_pred_uniform = [(np.ones(days_to_predict)/days_to_predict).round(5).tolist()] * len(val_dataset)

This is how a uniform distribution baseline output would look like

In [ ]:
pd.DataFrame(y_pred_uniform, columns=range(1,days_to_predict+1)).head()

##### How the inventory_days probability distribution looks like for a random observation 

In [ ]:
sku, stock,  days = pd.DataFrame(val_dataset)[['sku','target_stock','inventory_days']].sample(1).to_dict(orient='records')[0].values()
plt.ylim([0,0.05])
plt.axvline(days, color='r')
plt.title(f'sku:{sku}, target_stock:{stock},target days: {days}')
plt.bar(range(1,31), np.ones(days_to_predict)/days_to_predict, color='green')

plt.xlabel('Days')
plt.ylabel('Probs')
plt.legend(['Target days', 'Uniform Dist.'])
plt.show()

##### Now let's score this model's prediction

##### Scoring function:

In [ ]:
def ranked_probability_score(y_true, y_pred):
    """
    Input
        y_true: np.array of shape 30. 
        y_pred: np.array of shape 30. 
    """
    return ((y_true.cumsum(axis=1) - y_pred.cumsum(axis=1))**2).sum(axis=1).mean()

def scoring_function(y_true, y_pred):
    """
    Input
        y_true: List of Ints of shape Nx1. Contain the target_stock
        y_pred: List of float of shape Nx30. Contain the prob for each day
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_true_one_hot = np.zeros_like(y_pred, dtype=np.float)
    y_true_one_hot[range(len(y_true)), y_true-1] = 1
    return ranked_probability_score(y_true_one_hot, y_pred)

In [ ]:
uniform_score = scoring_function(y_true_val, y_pred_uniform)
print('Uniform model got a validation RPS of: ',uniform_score)

***In the public leaderboard this approach got a score of 5.07***

#### Baseline #2: Linear Model

As the uniform distributioin works so well, the idea is to slighly move the distribution toward the target day.
To do so we are going to use a very wide normal distribution.

In [ ]:
def generate_batch_predictions(model, x_test, batch_size=10000, processors=20):
    """Function usefull for paralellize inference"""
    pool = mp.Pool(processors)
    batches = batchify(x_test,batch_size)
    results = pool.imap(model.predict_batch,batches)
    pool.close()
    output = []
    for r in tqdm(results, total=int(len(x_test)/batch_size), desc='generating preds...'):
        output.extend(r)
    preds_dict = {}
    for sku,probs in tqdm(output):
        preds_dict[sku] = probs
    y_pred = []
    for x in tqdm(x_test):
        pred = preds_dict[x['sku']]
        y_pred.append(pred)
    return y_pred


def batchify(iterable, batch_size):
    """Convert an iterable in a batch-iterable"""
    iterator = iter(iterable)
    for first in iterator:
        yield list(chain([first], islice(iterator, batch_size - 1)))


In [ ]:
from scipy.stats import norm
step=1

In [ ]:
model_ = norm(15, 10)

In [ ]:
if step >= 1:
    x_axis = np.arange(-10, 40, 0.001)
    plt.plot(x_axis, model_.pdf(x_axis))
    plt.legend(['Normal dist'])

if step >= 2:    
    plt.axvline(0, color='black')
    plt.axvline(30, color='black')
    
if step >= 3:
    for i in range(30):
        plt.vlines(i,ymin=0,ymax=model_.pdf(i))

if step >= 4:
    scale = model_.cdf(30) - model_.cdf(0)
    x_axis = np.arange(0, 31, 1)
    plt.plot(x_axis, model_.pdf(x_axis)/scale)
    step = 0
step += 1
plt.show()

##### Model definition

In [ ]:
from scipy.stats import norm
from tqdm import tqdm

class LinearModel():
    """
    Linear model based on sold_quantity
    """
    def __init__(self, 
                 last_n_days=None, 
                 normalize=True):
        
        self.normalize = normalize
        self.last_n_days = last_n_days
        self.border_cases = 0
        self.normal_cases = 0
        
    def fit(self, data):
        """ Store mean and std-dev for each SKU """
        
        if self.last_n_days != None:
            min_training_date = str((pd.to_datetime(data.date.max())-timedelta(days=self.last_n_days)).date())
        else:
            min_training_date = str(data.date.min().date())
            
        self.parameters = (data[data.date >= min_training_date]
                           .groupby('sku')
                           .agg({'sold_quantity':['mean', 'std']})
                           .sold_quantity
                           .to_dict())

        self.general_mean = data.sold_quantity.mean()
        self.general_std = data.sold_quantity.std()
        return self 
    
    def calc_probs(self, norm_dist):
        #cut probs in days
        probs = []
        for i in range(1, 31):
            probs.append(norm_dist.cdf(i+1) - norm_dist.cdf(i))
        
        #if prob is zero, replace with uniform
        if np.sum(probs) == 0:
            return np.ones(30) / 30

        if self.normalize:
            probs = probs / np.sum(probs)
        return probs
    
    def predict(self, idx, stock):
        """ calculate mean and variance to stockout for a given SKU """
        #retrieve the mean and variance for the SKU
        if self.parameters['mean'].get(idx, 0.) != 0.:
            mean = self.parameters['mean'][idx]
            std = self.parameters['std'][idx]        
            self.normal_cases += 1
        else:
            #to catch border cases where there is no data in train or has all 0s.
            mean = self.general_mean
            std = self.general_std    
            self.border_cases += 1
            
        if std == 0. or np.isnan(std):
            std = self.general_std
        
        #convert quantities into days
        days_to_stockout = stock / mean
        std_days = (std / mean) * days_to_stockout
        return days_to_stockout, std_days
    
    def predict_proba(self, idx, stock):
        """ Calculates the 30 days probs given a SKU and a target_stock """
        days_to_stockout, std_days = self.predict(idx, stock)
        norm_dist = norm(days_to_stockout, std_days)
        return self.calc_probs(norm_dist)
    
    def predict_batch(self, X, proba=True):
        """ 
        Predict probs for many SKUs 
        Input:
            X: List of Dicts with keys sku and target_stock
        """
        result = []
        for x in X:
            idx = x['sku']
            stock = x['target_stock']
            if proba:
                result.append((idx, self.predict_proba(idx, stock)))
            else:
                result.append((idx, self.predict(idx, stock)))
        return result

##### Model Training

In [ ]:
%%time
model = LinearModel(last_n_days=14, normalize=True)

#train the model with train data
model.fit(data_train)

##### Inference

In [ ]:
y_pred_normal = generate_batch_predictions(model, val_dataset, batch_size=10000, processors=20)

##### How the inventory_days probability distribution looks like for a random observation in this case

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(8, 6), dpi=80)

sku, stock,  days = pd.DataFrame(val_dataset)[['sku','target_stock','inventory_days']].sample(1).to_dict(orient='records')[0].values()
probs = model.predict_proba(sku, stock)
mean_to_stockout, var_to_stockout = model.predict(sku, stock)
plt.bar(range(1,31), probs)
plt.axvline(days, color='r')
plt.title('sku:{}, target_stock:{}, mean: {}, std:{}'.format(int(sku), 
                                                             stock,
                                                             round(mean_to_stockout), 
                                                             round(var_to_stockout)))
plt.axhline(1/30, color='y')
plt.show()

In [ ]:
#calculate the score
normal_score = scoring_function(y_true_val, y_pred_normal)
print('Normal distribution model got a validation RPS of: ',normal_score)

### 5. Error analysis

In [ ]:
val_dataset_pd = pd.DataFrame(val_dataset)
scores = []
for y_t, y_p in tqdm(zip(val_dataset_pd['inventory_days'].to_list(), y_pred_normal)):
    scores.append(scoring_function(np.array([int(y_t)]), np.array([y_p])))
val_dataset_pd.loc[:, 'score'] = scores


In [ ]:
plt.scatter(val_dataset_pd.iloc[:10000].inventory_days, val_dataset_pd.iloc[:10000].score)
plt.xlabel('Days')
plt.ylabel('Score')
plt.title('Score by days')
plt.show()

Here we see ....

### 6. Train model to submit

Now that we have validated that the approach works, we train the model with all the data in order to make a submission

In [ ]:
all_data = pd.concat([data_train,data_val])

In [ ]:
model = LinearModel(last_n_days=14, normalize=True)

model.fit(all_data) #   <---- HERE WE TRAIN THE MODEL WITH FULL DATA !!!!

##### Generate predictions on test data

In [ ]:
x_test = data_test.reset_index()[['index','sku','target_stock']].to_dict(orient='records')

y_pred = generate_batch_predictions(model, x_test, batch_size=10000, processors=20)

##### Finally we generate a submission file with the model predictions

In [ ]:
def array2text(y_pred):
    """convert a list of number in a list of texts with 4 decimal positions """
    result = []
    for xs in tqdm(y_pred):
        line = []
        for x in xs:
            line.append('{:.4f}'.format(x))
        result.append(line)
    return result

def make_submission_file(y_pred, file_name='submission_file', compress=True, single_row=True):
    """Convert a list of text into a submition file"""
    result = array2text(y_pred)
    if compress:
        if single_row:
            file_name = f'{file_name}.csv.gz'
            with gzip.open(file_name, "wt") as f:
                writer = csv.writer(f)
                for row in tqdm(result, desc='making file...'):
                    writer.writerow(row)
        else:
            file_name = f'{file_name}.csv.gz'
            with gzip.open(file_name, "wt") as f:
                writer = csv.writer(f)
                writer.writerows(result)
    else:
        if single_row:
            file_name = f'{file_name}.csv'            
            with open(file_name, "w") as f:
                writer = csv.writer(f)
                for row in tqdm(result, desc='making file...'):
                    writer.writerow(row)
        else:
            file_name = f'{file_name}.csv'
            with open(file_name, "w") as f:
                writer = csv.writer(f)
                writer.writerows(result)
    return file_name

def read_submission_file(file_name, compress=False):
    if compress:
        with gzip.open(file_name, 'rt') as f:
            submission = f.read()
    else:
        with open(file_name, 'r') as f:
            submission = f.read()

In [ ]:
file_name = make_submission_file(y_pred, 'submittion_file_linear_model', compress=True, single_row=True)
print(f'Submission file created at: {file_name}')